<a href="https://colab.research.google.com/github/venkatrajg/SLM_fundamentals/blob/main/O'Reilly_SLM_Part1_Fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Small Language Models - Part 1 - Fundamentals

In [ ]:
# install required libraries
!pip install transformers accelerate torch sentencepiece bitsandbytes --quiet
print("Libraries installed!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 30.3 MB/s eta 0:00:00
Libraries installed!


In [ ]:
# Log in to Hugging Face
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
# define model ID
model_id_llama = "meta-llama/Llama-3.2-1B-Instruct"

# load tokenizer
tokenizer_llama = AutoTokenizer.from_pretrained(model_id_llama)

# load model
model_llama = AutoModelForCausalLM.from_pretrained(
    model_id_llama,
    torch_dtype=torch.bfloat16,
    load_in_4bit=True,
    device_map="auto"
)
print("Llama 3.2 1B Instruct loaded")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Llama 3.2 1B Instruct loaded


In [ ]:
# llama verification
print("Running quick verification...")
_prompt = "Hi! How are you today?" # define a simple input string
# convert the prompt text to tokens, make it a pytorch tensor,
# and send it to the same device the model is on (the GPU)
_inputs = tokenizer_llama(_prompt, return_tensors="pt").to(model_llama.device)

# ask the model to generate a response based on the input
# max_new_tokens limits the output length to 5 tokens for quick check
_outputs = model_llama.generate(**_inputs, max_new_tokens=5)

# decode the numeric output tokens back into readable text
# outputs[0] ges the first sequence in the batch
print("Llama Verification Ok:", tokenizer_llama.decode(_outputs[0]))

# cleaning up temporary variables
del _prompt, _inputs, _outputs

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Running quick verification...
Llama Verification Ok: <|begin_of_text|>Hi! How are you today? I hope you're having


In [ ]:
!nvidia-smi

Wed May 28 11:22:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   50C    P0             29W /   72W |    1745MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# --- Load Gemma 2 2B ---
model_id_gemma = "google/gemma-2-2b-it"
tokenizer_gemma = AutoTokenizer.from_pretrained(model_id_gemma)
model_gemma = AutoModelForCausalLM.from_pretrained(
    model_id_gemma,
    torch_dtype=torch.bfloat16,
    load_in_4bit=True,
    device_map="auto"
)
print(f"Gemma 2 2B loaded")

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Gemma 2 2B loaded


In [ ]:
# gemini verification
print("Running quick verification...")
_prompt = "Hi! How are you today?"
_inputs = tokenizer_gemma(_prompt, return_tensors="pt").to(model_gemma.device)
_outputs = model_gemma.generate(**_inputs, max_new_tokens=5)

print("Gemma Verification Ok:", tokenizer_gemma.decode(_outputs[0]))

# cleaning up temporary variables
del _prompt, _inputs, _outputs

Running quick verification...
Gemma Verification Ok: <bos>Hi! How are you today? 😊 

I'


In [ ]:
!nvidia-smi

Wed May 28 11:23:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   56C    P0             30W /   72W |    4587MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# --- Load Qwen 2.5 0.5B ---
model_id_qwen = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer_qwen = AutoTokenizer.from_pretrained(model_id_qwen)
model_qwen = AutoModelForCausalLM.from_pretrained(
    model_id_qwen,
    torch_dtype=torch.bfloat16,
    load_in_4bit=True,
    device_map="auto"
)
print(f"Qwen 2.5 0.5B loaded")

# qwen verification
print("Running quick verification...")
_prompt = "Hi! How are you today?"
_inputs = tokenizer_qwen(_prompt, return_tensors="pt").to(model_qwen.device)
_outputs = model_qwen.generate(**_inputs, max_new_tokens=5)
print("Qwen Verification Ok:", tokenizer_qwen.decode(_outputs[0]))

# cleaning up temporary variables
del _prompt, _inputs, _outputs

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Qwen 2.5 0.5B loaded
Running quick verification...
Qwen Verification Ok: Hi! How are you today? I'm a human,


In [ ]:
!nvidia-smi

Wed May 28 11:24:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   58C    P0             30W /   72W |    4633MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import time # Import the time library

# Define a function to handle text generation and timing
def generate_text(model, tokenizer, prompt, max_new_tokens):

    start_time = time.time() # Record the time before generation starts

    # Prepare the input: tokenize, convert to PyTorch tensors, move to GPU
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate text using the model
    outputs = model.generate(
        **inputs,                     # Pass the tokenized inputs
        max_new_tokens=max_new_tokens, # Set maximum number of new tokens to generate
    )
    end_time = time.time() # Record the time after generation finishes

    # Process the output
    output_ids = outputs[0] # Get the full sequence of token IDs
    input_token_len = inputs.input_ids.shape[1] # Find length of original input tokens
    generated_ids = output_ids[input_token_len:] # Isolate the newly generated token IDs
    generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True) # Convert generated IDs to text
    num_generated_tokens = len(generated_ids) # Count how many tokens were generated

    # Calculate and display performance
    duration = end_time - start_time
    tokens_per_sec = num_generated_tokens / duration
    print(f"Generated {num_generated_tokens} tokens in {duration:.2f} seconds ({tokens_per_sec:.2f} tokens/sec)")

    # Print the final generated text
    print("Output:")
    print(generated_text)

    # Return the text and speed for potential later use
    return generated_text, tokens_per_sec

In [ ]:
prompt = "Describe the year-round climate in London, England"

In [ ]:
# Generate with Llama 3.2 1B
llama_output, llama_speed = generate_text(model_llama, tokenizer_llama, prompt, max_new_tokens=500)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated 186 tokens in 5.69 seconds (32.70 tokens/sec)
Output:
.

London is a relatively mild city, with a temperate maritime climate. The year-round climate in London is characterized by:

* Mild winters: Winters in London are generally mild, with average temperatures ranging from 2°C to 10°C (36°F to 50°F) from December to February.
* Cool summers: Summers in London are cool, with average temperatures ranging from 10°C to 20°C (50°F to 68°F) from June to August.
* Rainy seasons: London experiences a rainy season from October to December, with average rainfall increasing from October to November.
* Overcast skies: London's climate is often overcast, with an average of 10-15 cloudy days per year, especially in the winter months.

Overall, the year-round climate in London is generally mild, with moderate temperatures and occasional rain showers. This makes London an attractive destination for tourists and residents alike.


In [ ]:
# Generate with Gemma 2 2B
gemma_output, gemma_speed = generate_text(model_gemma, tokenizer_gemma, prompt, max_new_tokens=500)

Generated 376 tokens in 26.39 seconds (14.25 tokens/sec)
Output:
.

London's climate is generally temperate and mild, with a distinct four-season pattern. Here's a breakdown:

**Spring (March-May):**
* Temperatures gradually increase, ranging from 10°C to 15°C (50°F to 59°F).
* Rainfall is moderate, with occasional showers.
* Days become longer and brighter.

**Summer (June-August):**
* Warm and sunny days with temperatures reaching 20°C to 25°C (68°F to 77°F).
* Rainfall is low, with occasional thunderstorms.
* Days are long and bright.

**Autumn (September-November):**
* Temperatures gradually decrease, with average highs around 15°C (59°F).
* Rainfall increases, with occasional heavy downpours.
* Days are shorter and cooler.

**Winter (December-February):**
* Cold and wet with temperatures averaging around 5°C (41°F).
* Rainfall is high, with frequent snowfalls.
* Days are short and dark.

**Overall:**
* London experiences a temperate climate with mild temperatures throughout the ye

In [ ]:
# Generate with Qwen 2.5 0.5B
qwen_output, qwen_speed = generate_text(model_qwen, tokenizer_qwen, prompt, max_new_tokens=500)

Generated 500 tokens in 22.76 seconds (21.97 tokens/sec)
Output:
. London has a warm and humid climate throughout the year.
- Winter: Snowmarch (December – February), March, May, June, July, August, September, October, November, December
- Summer: Temperatures range from 10–25°C, with an average temperature of around 24°C by late spring.
- Rain: Heavy downpour days are common during summer months, while most areas experience moderate to heavy rainfall in autumn.
- Precipitation: The city receives more than 3600mm of rain annually. This is one of the highest rainfall rates in Europe. 
- Humidity: The air quality can be very high in winter, which is often referred to as "foggy" weather. In summer, it's more humid, but less extreme. 
- Temperature: The city averages between -5°C and 30°C. The lowest temperatures occur in January and February, and the highest temperatures reach up to +30°C in August and September. 
- Seasons: The city experiences four distinct seasons. In general, the cold

In [ ]:
prompt_2 = """Scenario:
Four colored cups – Red, Blue, Green, and Yellow – are arranged in a circle.
One cup has a hidden star.

Facts:

The cup with the star is not Red.
The cup with the star is directly next to the Blue cup.
The Green cup is directly between the Yellow cup and the Red cup (when going around the circle in one direction).
Question:
Which color cup has the star? Just give the correct answer"""

In [ ]:
# Generate with Llama 3.2 1B
llama_output, llama_speed = generate_text(model_llama, tokenizer_llama, prompt_2, max_new_tokens=500)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated 6 tokens in 0.23 seconds (26.35 tokens/sec)
Output:
.

Answer:
Blue.


In [ ]:
# Generate with Gemma 2 2B
gemma_output, gemma_speed = generate_text(model_gemma, tokenizer_gemma, prompt_2, max_new_tokens=500)

Generated 9 tokens in 0.66 seconds (13.68 tokens/sec)
Output:
.

**Answer:** Yellow 



In [ ]:
# Generate with Qwen 2.5 0.5B
qwen_output, qwen_speed = generate_text(model_qwen, tokenizer_qwen, prompt_2, max_new_tokens=500)

Generated 321 tokens in 14.53 seconds (22.09 tokens/sec)
Output:
. To determine which color cup has the hidden star, let's analyze the given facts step by step:

1. **Fact 1**: The cup with the star is not Red.
2. **Fact 2**: The cup with the star is directly next to the Blue cup.
3. **Fact 3**: The Green cup is directly between the Yellow cup and the Red cup.
4. **Fact 4**: The Green cup is also directly between the Yellow cup and the Red cup.

From Fact 2, we know that blue is next to red. From Fact 4, we can deduce that yellow must be directly after blue because it cannot go before blue since blue is already directly behind red. This leaves us with yellow being directly after blue.

So, from these four facts, we have established that the yellow cup must be right after the red cup. But we still need to confirm that there isn't another possibility for the blue cup.

**Fact 1** doesn't exclude any possibilities, so it doesn't affect our conclusion about the blue cup. Similarly, fact 3 

In [ ]:
prompt_3 = """Write a simple Python function called `calculate_area`
that accepts the length and width of a rectangle and returns the area."""

In [ ]:
# Generate with Llama 3.2 1B
llama_output, llama_speed = generate_text(model_llama, tokenizer_llama, prompt_3, max_new_tokens=500)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated 199 tokens in 6.17 seconds (32.27 tokens/sec)
Output:
 The area of a rectangle is the product of the length and width.

## Step 1: Define the function calculate_area
The function `calculate_area` should accept two parameters, `length` and `width`, which are the dimensions of the rectangle.

## Step 2: Calculate the area
To calculate the area of the rectangle, we need to multiply the length by the width.

## Step 3: Return the calculated area
The function should return the calculated area as the result.

## Step 4: Write the code
Here is the code for the `calculate_area` function:
```
def calculate_area(length, width):
    return length * width
```

The final answer is $\boxed{0}$, as the problem doesn't provide any specific values for the length and width, and the function `calculate_area` doesn't require any input parameters. The output of the function is simply the result of the multiplication operation, which is the area of the rectangle.


In [ ]:
# Generate with Gemma 2 2B
gemma_output, gemma_speed = generate_text(model_gemma, tokenizer_gemma, prompt_3, max_new_tokens=500)

Generated 202 tokens in 14.32 seconds (14.11 tokens/sec)
Output:


**Example Usage:**

```python
>>> calculate_area(5, 10)
50
```

**Explanation:**

1. **Function Definition:**
   - `def calculate_area(length, width):` defines a function named `calculate_area` that takes two parameters: `length` and `width`.

2. **Area Calculation:**
   - `area = length * width` calculates the area of the rectangle by multiplying the length and width.

3. **Return Value:**
   - `return area` returns the calculated area as a result.

**Example Usage:**
- `calculate_area(5, 10)` calls the function with a length of 5 and a width of 10.
- The function calculates the area (5 * 10 = 50) and returns the value 50.


Let me know if you'd like to explore other Python functions! 



In [ ]:
# Generate with Qwen 2.5 0.5B
qwen_output, qwen_speed = generate_text(model_qwen, tokenizer_qwen, prompt_3, max_new_tokens=500)

Generated 125 tokens in 5.61 seconds (22.30 tokens/sec)
Output:
 The input values are integers.

### Example

```python
calculate_area(5, 6)
# Output: 30

calculate_area(7, 8)
# Output: 56

calculate_area(10, 2)
# Output: 20
```

Note that there is no restriction on the precision or rounding for floating-point numbers in this exercise.
```python
def calculate_area(length, width):
    """
    This function takes two parameters, length and width,
    and calculates the area of a rectangle by multiplying them together.
    """
    return length * width
```


In [ ]:
prompt = "Write a short story about a time-traveling cat"

In [ ]:
# Define a function to handle text generation and timing
def generate_text(model, tokenizer, prompt, max_new_tokens, temperature):

    start_time = time.time() # Record the time before generation starts

    # Prepare the input: tokenize, convert to PyTorch tensors, move to GPU
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate text using the model
    outputs = model.generate(
        **inputs,                     # Pass the tokenized inputs
        max_new_tokens=max_new_tokens, # Set maximum number of new tokens to generate
        temperature=temperature,      # Control the randomness (creativity vs focus)
        do_sample=True,               # Enable sampling (needed for temperature)
        #pad_token_id=tokenizer.eos_token_id # Prevent warnings about padding token
    )
    end_time = time.time() # Record the time after generation finishes

    # Process the output
    output_ids = outputs[0] # Get the full sequence of token IDs
    input_token_len = inputs.input_ids.shape[1] # Find length of original input tokens
    generated_ids = output_ids[input_token_len:] # Isolate the newly generated token IDs
    generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True) # Convert generated IDs to text
    num_generated_tokens = len(generated_ids) # Count how many tokens were generated

    # Calculate and display performance
    duration = end_time - start_time
    tokens_per_sec = num_generated_tokens / duration
    print(f"Generated {num_generated_tokens} tokens in {duration:.2f} seconds ({tokens_per_sec:.2f} tokens/sec)")

    # Print the final generated text
    print("Output:")
    print(generated_text)

    # Return the text and speed for potential later use
    return generated_text, tokens_per_sec

In [ ]:
# low temperature
_ = generate_text(model_gemma, tokenizer_gemma, prompt, max_new_tokens=500, temperature=0.2)

Generated 443 tokens in 31.03 seconds (14.27 tokens/sec)
Output:
 named Mittens.

* * *

Mittens wasn't your average tabby. He wasn't content with napping in sunbeams or chasing dust bunnies. No, Mittens was a time-traveling feline, his fur imbued with a peculiar, shimmering energy that allowed him to leap through time and space. His adventures were legendary in the feline community, whispered about in hushed tones at the back of the alley.

One day, Mittens found himself in ancient Egypt. The air was thick with the scent of incense and the rhythmic clang of metal on metal. He watched, fascinated, as a pharaoh, adorned in gold and jewels, was being carried on a grand procession. He yearned to join them, to feel the weight of history on his fur.

He slipped into the procession unnoticed, his sleek black fur blending seamlessly with the shadows. He watched as the pharaoh, his face obscured by a heavy mask, was being presented to the gods. The air crackled with divine power, and the groun

In [ ]:
# high temperature
_ = generate_text(model_gemma, tokenizer_gemma, prompt, max_new_tokens=500, temperature=1.0)

Generated 500 tokens in 35.17 seconds (14.22 tokens/sec)
Output:
, Sir Whiskers.

* **Disclaimer:** My intent is to provide a comedic and fantastical story, aiming for lightheartedness and entertainment. Please be aware that this is purely fictional and does not reflect the reality of time travel.**

Sir Whiskers, a Persian with the fluffiest tail you've ever seen, wasn't just your average feline. He possessed an advanced time-traveling collar, a gift from his eccentric inventor uncle, "Gadget Gene." This collar allowed Sir Whiskers to hop across time to any era he wished, be that a medieval joust or a futuristic rave. He spent his days, well, *exploding into the wrong decade* as if it were an exciting, chaotic trip to the playground.

He was currently trying out his time-traveling device, disguised as a miniature cat statue, a bit of "DIY genius" as he liked to call it. He was perched atop a dusty, old filing cabinet. 

There, he was transported to... a cozy antique shop. 

"Excuse me